In [1]:
import sys  
sys.executable

'/Users/zed/.pyenv/versions/anaconda3-2021.05/bin/python3'

In [2]:
import os
import torch
import torch.nn as nn
import torch.utils.data as Data
from sklearn.preprocessing import MinMaxScaler
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import talib
import plotly.graph_objects as go


In [3]:
dataset0 = pd.read_csv('/Users/zed/AI_Lab/DoubleEnsembleML/report/mybtc.csv',index_col= 0)

In [17]:
Y = dataset0["tag"].values.reshape(-1,1)

In [18]:
Y = Y.squeeze(1).reshape(-1,1)

In [19]:
X = dataset0.drop(["Date","tag"],axis=1).dropna(how = "any")

In [20]:
X 

,High,Low,Close,Volume,Open,H-L,3day MA,10day MA,30day MA,CCI,RSI,ATR,OBV,HT_DCPERIOD,Last1Price,Last2Price,Last3Price,Last4Price,Last5Price
32,730.0000,680.01000,699.30255,31.042254,714.12206,49.99000,761.410300,796.146558,819.809884,-78.571429,22.702711,40.886460,-168.377924,15.646696,714.12206,767.60884,802.50000,802.40000,819.78000
33,756.0998,684.85098,689.00000,38.291274,699.30255,71.24882,727.011150,786.576813,815.057658,-78.571429,21.318570,45.223940,-206.669198,17.190252,699.30255,714.12206,767.60884,802.50000,802.40000
34,748.0000,550.00000,690.20000,29.856668,689.00000,198.00000,700.808203,774.676813,809.446639,-100.000000,21.942173,67.049091,-176.812530,18.731589,689.00000,699.30255,714.12206,767.60884,802.50000
35,720.0000,660.00000,714.56111,33.331274,690.20000,60.00000,692.834183,762.477776,802.458604,-57.142857,33.905912,66.042078,-143.481256,20.302886,690.20000,689.00000,699.30255,714.12206,767.60884
36,709.9300,651.30901,670.66590,17.641851,714.56111,58.62099,697.920370,751.433887,797.610641,-57.142857,25.868810,65.643510,-161.123107,21.888230,714.56111,690.20000,689.00000,699.30255,714.12206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2652,56805.5000,53600.00000,53815.20000,4788.013532,56500.00000,3205.50000,56153.266667,59957.890000,57851.923333,-28.571429,31.587568,3594.414995,606743.431869,15.992169,56500.00000,55720.00000,56239.80000,60085.20000,61455.00000
2653,55459.4000,50500.00000,51730.00000,10144.204074,53815.20000,4959.40000,55345.066667,59343.810000,57842.593333,-57.142857,26.873534,3789.412853,596599.227795,15.760320,53815.20000,56500.00000,55720.00000,56239.80000,60085.20000
2654,52124.4000,47549.30000,51178.70000,11159.243380,51730.00000,4575.10000,54015.066667,58534.440000,57755.216667,-64.285714,25.731360,3901.653874,585439.984415,15.631061,51730.00000,53815.20000,56500.00000,55720.00000,56239.80000
2655,51200.0000,48746.40000,50093.40000,4039.459093,51178.70000,2453.60000,52241.300000,57293.590000,57718.046667,-64.285714,23.517675,3694.789035,581400.525322,15.618939,51178.70000,51730.00000,53815.20000,56500.00000,55720.00000


In [21]:
def sliding_windows(data, label, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = label[i+seq_length,:]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [48]:
ystr = []
for i in Y:
    ystr.append(str(i[0]))
ystr = np.array(ystr)

In [59]:
from sklearn.preprocessing import MinMaxScaler


x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

#converting dataset into x_train and y_train
X = x_scaler.fit_transform(X)
Y = y_scaler.fit_transform(Y.reshape(-1,1))


In [60]:
x, y = sliding_windows(X, Y, 15)

In [61]:
i = 0.8
y_train,y_test = y[:int(y.shape[0]*i)],y[int(y.shape[0]*i):]
x_train,x_test = x[:int(x.shape[0]*i)],x[int(x.shape[0]*i):]

# lstm: seq, batch, feature
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataX = torch.Tensor(x.transpose(1,0,2))
dataY = torch.Tensor(y)
trainX = torch.Tensor(x_train.transpose(1,0,2))
trainY = torch.Tensor(y_train)
testX = torch.Tensor(x_test.transpose(1,0,2))
testY = torch.Tensor(y_test)
trainX.shape, trainY.shape

(torch.Size([15, 2087, 19]), torch.Size([2087, 1]))

In [62]:
class LSTM(nn.Module):
    
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers)
        
        self.fc = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        #不手动指定 h 和 c 的话，默认就是 0
        # h_0 = torch.zeros(
        #     self.num_layers, x.size(1), self.hidden_size)
        
        # c_0 = torch.zeros(
        #     self.num_layers, x.size(0), self.hidden_size)
        
        # #Propagate input through LSTM
        # ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        ula, (h_out, _) = self.lstm(x)
        h_out = h_out.view(-1, self.hidden_size)

        
        out = self.fc(h_out)
        out = self.fc2(out)
        out = self.sig(out)
        return out

In [76]:
num_epochs = 1000
learning_rate = 0.00002

input_size = X.shape[1] # The number of expected features in the input x
hidden_size = 100        # The number of features in the hidden state h
num_layers = 1 # Number of recurrent layers.
seq_length = 15
num_classes = 3 # output

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate, momentum=0.09)
# Train the model
lstm.train()
lstm.to(device)
trainX = trainX.to(device)
for epoch in tqdm(range(num_epochs)):
    optimizer.zero_grad()
    outputs = lstm(trainX)
    # obtain the loss function
    loss = criterion(outputs, trainY.long().squeeze(1))
    
    loss.backward()
    
    optimizer.step()
    if epoch%200 == 0 :
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

  0%|          | 1/1000 [00:00<05:01,  3.31it/s]

Epoch: 0, loss: 1.09684


 20%|██        | 201/1000 [00:53<03:22,  3.94it/s]

Epoch: 200, loss: 1.09674


 40%|████      | 401/1000 [01:46<02:48,  3.55it/s]

...

 60%|██████    | 601/1000 [02:51<01:57,  3.41it/s]

Epoch: 600, loss: 1.09653


 80%|████████  | 801/1000 [03:48<00:49,  4.03it/s]

Epoch: 800, loss: 1.09643


100%|██████████| 1000/1000 [04:40<00:00,  3.56it/s]


In [67]:
_, predicted = torch.max(outputs, 1)


In [74]:
dataY_pred = lstm(dataX)
ypr = dataY_pred.data.numpy().argmax(axis=1)

In [113]:
lstm.eval()
lstm.to(torch.device('cpu'))
with torch.no_grad():
    dataY_pred = lstm(dataX)

dataY_pred = dataY_pred.data.numpy().argmax(axis=0)
dataY_truth = dataY.data.numpy()

dataY_pred = y_scaler.inverse_transform(dataY_pred)
dataY_truth = y_scaler.inverse_transform(dataY_truth)


fig = go.Figure(go.Scatter(y=dataY_truth.flatten(),name='Ground Truth'))
fig.add_trace(go.Scatter(y=dataY_pred.flatten(),name='Predicted'))

fig.update_layout(
    shapes = [dict(
        x0=len(x_train), x1=len(x_train), y0=-1, y1=1, xref='x', yref='paper',
        line_width=2)], #在图上划分训练集和测试集
    xaxis_rangeslider_visible=True,
)



In [123]:
lstm.eval()
lstm.to(torch.device('cpu'))
with torch.no_grad():
    dataY_pred = lstm(dataX)

dataY_pred = dataY_pred.data.numpy()
dataY_truth = dataY.data.numpy()

In [124]:
dataY_pred

array([[ 0.6050823 ,  0.03275242, -0.78030384],
       [ 0.60568273,  0.03279406, -0.78132534],
       [ 0.60462445,  0.0315816 , -0.77727735],
       ...,
       [ 0.58448297,  0.0626955 , -0.8643252 ],
       [ 0.5815581 ,  0.06026142, -0.8556609 ],
       [ 0.57697755,  0.06053086, -0.8474711 ]], dtype=float32)

0

AttributeError: 'numpy.ndarray' object has no attribute 'maxid'